In [ ]:
import matplotlib.pyplot as plt
import torch
from alse.utils import identify_samples_which_satisfy_constraints, normalize, unnormalize
from mpl_toolkits import mplot3d
from alse.utils import read_excel
from alse.alse import alse
from alse.accuracy import gen_test_point_for_accuracy
import plotly.graph_objects as go
import plotly.express as px
tkwargs = {
    "device": torch.device("cpu"),
    "dtype": torch.float,
}

X, width, pow_cap, wth = read_excel("../test_data/8_12_data.xlsx",["P (W)", "V (mm/min)", "spot size (mm)"], ["width (mm)", "powder_cap", "wth"])
bounds = torch.tensor([[900, 600, 1], [2700, 2200, 4]])
constraints = [("gt", 2.5), ("gt", 0.65), ("gt", 6)]

algo = alse(X, bounds, [width, pow_cap, wth], constraints)

algo.initialize_model(["reg", "reg", "reg"])

algo.next_test_points(1)

In [ ]:
%matplotlib widget

In [ ]:
N = 50

Normalized grid

In [ ]:

Xplt, Yplt, Zplt = torch.meshgrid(
    torch.linspace(0, 1, N, **tkwargs), 
    torch.linspace(0, 1, N, **tkwargs),
    torch.linspace(0, 1, N, **tkwargs),
    indexing='xy',
)
xplt = torch.stack(
(
    torch.reshape(Xplt, (N**3,)),
    torch.reshape(Yplt, (N**3,)),
    torch.reshape(Zplt, (N**3,)),
),
dim=1,
)

In [ ]:
a = algo.list_of_models[0](xplt).loc.detach() > 2.5
b = algo.list_of_models[1](xplt).loc.detach() > 0.65
c = algo.list_of_models[2](xplt).loc.detach() > 6
pos_overlap = (a & b & c).unsqueeze(-1)

Actual grid of parameter space

In [ ]:

Xplt, Yplt, Zplt = torch.meshgrid(
    torch.linspace(900, 2700, N, **tkwargs), 
    torch.linspace(600, 2200, N, **tkwargs),
    torch.linspace(1, 4, N, **tkwargs),
    indexing='xy',
)
xplt = torch.stack(
(
    torch.reshape(Xplt, (N**3,)),
    torch.reshape(Yplt, (N**3,)),
    torch.reshape(Zplt, (N**3,)),
),
dim=1,
)

In [ ]:
in_bound_tpoints, out_of_bound_tpoints, pos_surface = gen_test_point_for_accuracy(pos_overlap, 
                                                                                  candidates=xplt, 
                                                                                  num_points=10, 
                                                                                  N=N, 
                                                                                  threshold=0.1)

In [ ]:
fig = go.Figure(data=[go.Scatter3d(z=pos_surface[:,2], x=pos_surface[:,0], y=pos_surface[:,1],mode='markers')])
fig.update_traces(marker_size = 2)
fig.update_layout(
    autosize=False,
    width=1000,
    height=1000,
    title = go.layout.Title(text='Powder Capture %'),
    scene=dict(xaxis_title='Power', 
                yaxis_title='Velocity',
                zaxis_title='Spot Size')
)
fig.show()

In [ ]:
in_bound_tpoints

In [ ]:
out_of_bound_tpoints

In [ ]:
'''
Uncomment to verify the threshold is working
Need to add new_x to the output of gen_test_point_for_accuracy
'''
# fig = go.Figure(data=[go.Scatter3d(z=new_x[:,2], x=new_x[:,0], y=new_x[:,1],mode='markers')])
# fig.update_traces(marker_size = 1)
# fig.update_layout(
#     autosize=False,
#     width=1000,
#     height=1000,
#     title = go.layout.Title(text='Powder Capture %'),
#     scene=dict(xaxis_title='Power', 
#                 yaxis_title='Velocity',
#                 zaxis_title='Spot Size')
# )
# fig.show()

Ignore everything below for now

In [ ]:
'''
Generate a 360 step camera coordinate list
'''
import math
dist = 2
step = 2*math.pi/360
angle = 0
x = []
y = []
while angle < 2*math.pi:
    x.append(dist*math.cos(angle))
    y.append(dist*math.sin(angle))
    angle +=step

In [ ]:

Xplt, Yplt, Zplt = torch.meshgrid(
    torch.linspace(0, 1, N, **tkwargs), 
    torch.linspace(0, 1, N, **tkwargs),
    torch.linspace(0, 1, N, **tkwargs),
    indexing='xy',
)
xplt = torch.stack(
(
    torch.reshape(Xplt, (N**3,)),
    torch.reshape(Yplt, (N**3,)),
    torch.reshape(Zplt, (N**3,)),
),
dim=1,
)
Xplt, Yplt, Zplt = torch.meshgrid(
    torch.linspace(900, 2700, N, **tkwargs), 
    torch.linspace(600, 2200, N, **tkwargs),
    torch.linspace(1, 4, N, **tkwargs),
    indexing='xy',
)

In [ ]:
values = algo.list_of_models[2](xplt).loc.detach()
values.max()

In [ ]:
'''
This is for generating a 360 view
Uncomment wirte_image and change range to 361 to
generate frames, otherwise skip
'''

for i in range(0):
    values = algo.list_of_models[2](xplt).loc.detach()
    fig = go.Figure(data=go.Isosurface(
        x=Xplt.flatten(),
        y=Yplt.flatten(),
        z=Zplt.flatten(),
        value=values,
        isomin=4,
        isomax=22,
        surface_count=7, # number of isosurfaces, 2 by default: only min and max
        colorbar_nticks=7, # colorbar ticks correspond to isosurface values
        caps=dict(x_show=False, y_show=False)
        ))

    camera = dict(
        eye=dict(x=x[i], y=y[i], z=0.25)
    )

    fig.update_layout(scene_camera=camera,
        autosize=False,
        width=1200,
        height=800,
        title = go.layout.Title(text='Width-to-height ratio'),
        scene=dict(xaxis_title='Power', 
                   yaxis_title='Velocity',
                   zaxis_title='Spot Size')
        )
    # fig.show()
    fig.write_image(f'./images/{i}.png', engine='orca')

In [ ]:
in_bound_tpoints

In [ ]:
out_of_bound_tpoints

In [ ]:
a = in_bound_tpoints
b = out_of_bound_tpoints

In [ ]:
'''
This is for generating a 360 view
Uncomment wirte_image and change range to 361 to
generate frames, otherwise skip
'''

for i in range(1):
    values = pos_overlap.float() # convert to float otherwise cannot show
    fig = go.Figure(data=[go.Scatter3d(x=a[:,0], y=a[:,1], z=a[:,2],mode='markers', marker=dict(color='green')),
                          go.Scatter3d(x=b[:,0], y=b[:,1], z=b[:,2],mode='markers', marker=dict(color='red')),
        go.Isosurface(
        x=Xplt.flatten(),
        y=Yplt.flatten(),
        z=Zplt.flatten(),
        value=values,
        isomin=0,
        isomax=1,
        surface_count=4, # number of isosurfaces, 2 by default: only min and max
        colorbar_nticks=4, # colorbar ticks correspond to isosurface values
        caps=dict(x_show=False, y_show=False)
        )])

    # camera = dict(
    #     eye=dict(x=x[i], y=y[i], z=0.25)
    # )

    fig.update_layout(
        autosize=False,
        width=1200,
        height=800,
        title = go.layout.Title(text='Powder Capture %'),
        scene=dict(xaxis_title='Power', 
                   yaxis_title='Velocity',
                   zaxis_title='Spot Size')
        )
    fig.show()
    # fig.write_image(f'./images/{i}.png', engine='orca')

In [ ]:
N = 50
Xplt, Yplt, Zplt = torch.meshgrid(
    torch.linspace(0, 1, N, **tkwargs), 
    torch.linspace(0, 1, N, **tkwargs),
    torch.linspace(0, 1, N, **tkwargs),
    indexing='xy',
)
xplt = torch.stack(
(
    torch.reshape(Xplt, (N**3,)),
    torch.reshape(Yplt, (N**3,)),
    torch.reshape(Zplt, (N**3,)),
),
dim=1,
)
Xplt, Yplt, Zplt = torch.meshgrid(
    torch.linspace(900, 2700, N, **tkwargs), 
    torch.linspace(600, 2200, N, **tkwargs),
    torch.linspace(1, 4, N, **tkwargs),
    indexing='xy',
)

In [ ]:
'''
Plot the posterior overlaps and generated test points
'''

values = algo.list_of_models[1](xplt).loc.detach()
values2 = pos_overlap.float()
df = a
something = []
fig = go.Figure(data=[go.Scatter3d(x=a[:,0], y=a[:,1], z=a[:,2],mode='markers', marker=dict(color='green')),
                      go.Scatter3d(x=b[:,0], y=b[:,1], z=b[:,2],mode='markers', marker=dict(color='red')),
                      go.Isosurface(
                      x=Xplt.flatten(),
                      y=Yplt.flatten(),
                      z=Zplt.flatten(),
                      value=values2,
                      isomin=0,
                      isomax=1,
                      surface_count=4, # number of isosurfaces, 2 by default: only min and max
                      colorbar_nticks=4, # colorbar ticks correspond to isosurface values
                      caps=dict(x_show=False, y_show=False)
                      )])

camera = dict(
    eye=dict(x=2, y=2, z=0.25)
)

fig.update_layout(scene_camera=camera,
    autosize=False,
    width=1400,
    height=1100,
    title = go.layout.Title(text='Powder Capture %'),
    scene=dict(xaxis_title='Power', 
                yaxis_title='Velocity',
                zaxis_title='Spot Size')
    )
fig.show()
# fig.write_image(f'./images/{i}.png', engine='orca')

In [ ]:
# acq_val = algo.get_acq_val_grid(25, 3).round(decimals=4).detach()

Posterior

Surface Plot

In [ ]:
# N1, N2 = 100, 100
# Xplt, Yplt = torch.meshgrid(
#     torch.linspace(0, 1, N1, **tkwargs), 
#     torch.linspace(0, 1, N2, **tkwargs),
#     indexing='xy',
# )
# xplt = torch.stack(
# (
#     torch.reshape(Xplt, (Xplt.shape[0] * Xplt.shape[1],)),
#     torch.reshape(Yplt, (Yplt.shape[0] * Yplt.shape[1],)),
#     torch.ones(N1*N2)*0/100
# ),
# dim=1,
# )
# xp, yp = torch.meshgrid(
#     torch.linspace(900, 2700, N1, **tkwargs), 
#     torch.linspace(600, 2200, N2, **tkwargs),
#     indexing='xy',
# )

In [ ]:
# for i in range(120):
#     spot_size = round(i*3/120+1, 1)

#     xplt = torch.stack(
#     (
#         torch.reshape(Xplt, (Xplt.shape[0] * Xplt.shape[1],)),
#         torch.reshape(Yplt, (Yplt.shape[0] * Yplt.shape[1],)),
#         torch.ones(N1*N2)*i/120
#     ),
#     dim=1,
#     )
    
#     # predicted_width = algo.list_of_models[0](xplt).loc.detach().reshape(100,100)
#     # predicted_pow_cap = algo.list_of_models[1](xplt).loc.detach().reshape(100,100)
#     predicted_wth = algo.list_of_models[2](xplt).loc.detach().reshape(100,100)
#     fig = plt.figure(figsize=(10, 10))
#     ax = plt.axes(projection='3d')

#     ax.set_xlabel("Power")
#     ax.set_ylabel("Velocity")
#     # ax.set_zlabel("Predicted Width")
#     ax.set_zlabel("Predicted wth")
#     img = ax.plot_surface(xp, yp, predicted_wth, cmap='coolwarm')
#     ax.set_title(f'Spot size: {spot_size}mm')
#     ax.set_xlim(900, 2700)
#     ax.set_ylim(600, 2200)
#     ax.set_zlim(4, 24)

#     # cbar = fig.colorbar(img, fraction=0.025)
#     # cbar.set_label('Width', rotation=270)
#     # plt.show()
#     plt.savefig(f'./img/{i}.png', dpi=300)
#     plt.close()